In [21]:
using PerlaTonettiWaugh, Plots, BenchmarkTools

In [17]:
z_min = 0.0 
z_max = 5.0
M = 1000
z_grid = range(z_min, stop = z_max, length = M) # Since we only care about the grid. 

# Time
tstops_min_Δ_val = 1e-3 # minimum distance between tstops to be used for DE solvers.
# Define common objects. 
d_0 = 5
d_T = 2.3701
params = (ρ = 0.02, σ = 4.2508, N = 10, θ = 5.1269, γ = 1.00, κ = 0.013, ζ = 1, η = 0, Theta = 1, χ = 1/(2.1868), υ = 0.0593, μ = 0, δ = 0.053, d = 2.3701) # Baselines per Jesse. 
stationary_sol = stationary_numerical(params, z_grid) # solution at t = T
println("g = $(stationary_sol.g), z_hat = $(stationary_sol.z_hat), Ω = $(stationary_sol.Ω)")

g = 0.020419880554626162, z_hat = 1.425712535487968, Ω = 1.0365494651388318


In [22]:
T = 5.0
settings = (z = z_grid, tstops = 0:T)
Ω(t) = stationary_sol.Ω
stationary_sol.Ω
Δ_E = 1e-4
E(t) = (log(Ω(t+Δ_E)) - log(Ω(t-Δ_E)))/(2*Δ_E) + params.δ
p = PerlaTonettiWaugh.get_p(params, stationary_sol, settings, Ω, T)
dae = PerlaTonettiWaugh.PTW_DAEProblem(params, stationary_sol, settings, E, Ω, T, p)

(dae_prob = DAEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (5.0, 0.0)
u0: [1.1868, 1.16807, 1.14991, 1.13229, 1.11521, 1.09864, 1.08257, 1.06699, 1.05188, 1.03723  …  0.737586, 0.730145, 0.722277, 0.713959, 0.705163, 0.695863, 0.68603, 0.675633, 0.0204199, 1.42571]
du0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], callback = DiffEqBase.DiscreteCallback{getfield(DiffEqCallbacks, Symbol("##28#29")),DiffEqCallbacks.SavingAffect{getfield(PerlaTonettiWaugh, Symbol("##57#63")){typeof(E),typeof(Ω),getfield(PerlaTonettiWaugh, Symbol("#get_S#58")){Int64,Float64,Float64},getfield(PerlaTonettiWaugh, Symbol("#get_L_tilde#59")){Float64,Float64,Float64,Int64,Int64}},Float64,Tuple{Float64,Float64},DataStructures.BinaryHeap{Float64,DataStructures.GreaterThan},Array{Float64,1}},typeof(DiffEqCallbacks.saving_initialize)}(getfield(DiffEqCallbacks, Symbol("##28#29"))(), DiffEqCallbacks.SavingAffect{getfield(Per

In [20]:
#Testing the f!
du = zeros(M)
u = [stationary_sol.v_tilde; stationary_sol.g; stationary_sol.z_hat]
resid = similar(u)
du = zeros(M+2)
t = 0.0
@btime dae.f!($resid,$du,$u,$p,$t)

  6.239 ms (126 allocations: 7.81 MiB)


1000-element view(::Array{Float64,1}, 1:1000) with eltype Float64:
 -9.016466739897311e-11 
 -9.016466739897311e-11 
 -9.016466739897311e-11 
 -9.01504565442579e-11  
 -9.01504565442579e-11  
 -9.01362456895427e-11  
 -9.016466739897311e-11 
 -9.01362456895427e-11  
 -9.01504565442579e-11  
 -9.01362456895427e-11  
 -9.01362456895427e-11  
 -9.01504565442579e-11  
 -9.01504565442579e-11  
  ⋮                     
 -1.3922836494817403e-10
 -1.3923437403029482e-10
 -1.392208431871822e-10 
 -1.392249371345855e-10 
 -1.3922608899097355e-10
 -1.3921717945120093e-10
 -1.3921867825228418e-10
 -1.3922184238790436e-10
 -1.3922427100077073e-10
 -1.392370108099783e-10 
 -1.3922771269214707e-10
 -1.39220135420004e-10  

1000-element Array{Float64,1}:
 1.186800000000357 
 1.168072362393182 
 1.1499093016976925
 1.1322940036233045
 1.115210166396562 
 1.098641985818218 
 1.0825741407943366
 1.0669917793286445
 1.051880504963822 
 1.0372263636598893
 1.0230158310982842
 1.009235800400677 
 0.9958735702519935
 ⋮                 
 0.7635286699344704
 0.7575748485995273
 0.7512797025406552
 0.7446236640762685
 0.7375860437360083
 0.7301449659501112
 0.7222773010519343
 0.7139585933822786
 0.7051629852720296
 0.6958631366668258
 0.6860301401439184
 0.675633431057062 